In [9]:
# import numpy as np 
# import h5py
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras import layers
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.initializers import Constant
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping

In [10]:
# model = Sequential()
# model.add(layers.Conv3D(32,(3,3,3),activation='relu',input_shape=(16,16,16,1),bias_initializer=Constant(0.01)))
# model.add(layers.Conv3D(32,(3,3,3),activation='relu',bias_initializer=Constant(0.01)))
# model.add(layers.MaxPooling3D((2,2,2)))
# model.add(layers.Conv3D(64,(3,3,3),activation='relu'))
# model.add(layers.Conv3D(64,(2,2,2),activation='relu'))
# model.add(layers.MaxPooling3D((2,2,2)))
# model.add(layers.Dropout(0.6))
# model.add(layers.Flatten())
# model.add(layers.Dense(256,'relu'))
# model.add(layers.Dropout(0.7))
# model.add(layers.Dense(128,'relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(10,'softmax'))
# model.summary()


In [12]:
# 3d-cnn implement
#importing libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
import os 

#for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt

#for creating validation set
from sklearn.model_selection import train_test_split
#for evaluating the model
from sklearn.metrics import accuracy_score

#Pytorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import h5py
#from plot3D import *


In [13]:
#process the dataset

In [15]:
#convert all the variables to pytorch tensor format
#X_train should have shape (num_of_dataset,50,100,29,3) and targets_train(num_of_dataset,1)
#X_test should have shape (num_of_test_dataset,50,100,29,3) and targets_test(num_of_test_dataset,1)
train_x=torch.from_numpy(X_train).float()#change the array into Tensor
train_y=torch.from_numpy(targets_train).long()
test_x=torch.from_numpy(X_test).float()
test_y=torch.from_numpy(targets_test).long()

NameError: ignored

In [16]:
batch_size=10#the batch_size we will use for the training

#pytorch train and test sets
train=torch.utils.data.TensorDataset(train_x,train_y)
test=torch.utils.data.TensorDataset(test_x,test_y)

#data loader
train_loader=torch.utils.data.DataLoader(train,batch_size=batch_size,shuffle=False)
test_loader=torch.utils.data.DataLoader(test,batch_size=batch_size,shuffle=False)#initialize the dataset and divide them into groups


NameError: ignored

In [17]:
#implement the model
num_classes=5
#50*100*29
#create CNN model
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel,self).__init__()

    self.conv_layer1=self._conv_layer_set(3,32)
    #50-2=48
    #100-2=98
    #29-2=27
    #24，49，13
    self.conv_layer2=self._conv_layer_set(32,64)
    #24-2=22
    #49-2=47
    #13-2=11
    #11，23，5
    self.fc1=nn.Linear(11*23*5*64, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3=nn.Linear(64,num_classes)
    self.relu = nn.LeakyReLU()
    self.batch=nn.BatchNorm1d(128)
    self.drop=nn.Dropout(p=0.15)        
        
  def _conv_layer_set(self, in_c, out_c):
        conv_layer = nn.Sequential(
        nn.Conv3d(in_c, out_c, kernel_size=(3, 3, 3), padding=0),
        nn.LeakyReLU(),
        nn.MaxPool3d((2, 2, 2)),
        )
        return conv_layer
    

  def forward(self, x):
        # Set 1
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.batch(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

#Definition of hyperparameters
n_iters = 2000
#num_epochs = n_iters / (len(train_x) / batch_size)
#num_epochs = int(num_epochs)

# Create CNN
model = CNNModel()
model.cuda()
print(model)

# Cross Entropy Loss 
error = nn.CrossEntropyLoss()

# SGD Optimizer
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

CNNModel(
  (conv_layer1): Sequential(
    (0): Conv3d(3, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer2): Sequential(
    (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=80960, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=5, bias=True)
  (relu): LeakyReLU(negative_slope=0.01)
  (batch): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop): Dropout(p=0.15, inplace=False)
)


In [18]:
#train model
# CNN model training
count = 0
loss_list = []
iteration_list = []
accuracy_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        train = Variable(images.view(10,3,50,100,29))
        labels = Variable(labels)
        # Clear gradients
        optimizer.zero_grad()
        # Forward propagation
        outputs = model(train)
        # Calculate softmax and ross entropy loss
        loss = error(outputs, labels)
        # Calculating gradients
        loss.backward()
        # Update parameters
        optimizer.step()
        
        count += 1
        if count % 50 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                
                test = Variable(images.view(10,3,50,100,29))
                # Forward propagation
                outputs = model(test)

                # Get predictions from the maximum value
                predicted = torch.max(outputs.data, 1)[1]
                
                # Total number of labels
                total += len(labels)
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / float(total)
            
            # store loss and iteration
            loss_list.append(loss.data)
            iteration_list.append(count)
            accuracy_list.append(accuracy)
        if count % 500 == 0:
            # Print Loss
            print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data, accuracy))


NameError: ignored